# Analyse de l'Impact de la Compression sur un Moteur de Recherche d'Images

Ce projet étudie l'influence de différentes méthodes de compression sur les performances d'un moteur de recherche d'images basé sur la similarité visuelle. L'analyse compare quatre approches : deux méthodes classiques (PNG sans perte et JPEG avec perte) et deux méthodes modernes (compression par ondelettes de Haar et transformation DCT).

Le dataset utilisé contient environ 3000 images de montagnes et de neige avec des features CNN pré-extraites (11520 dimensions). L'objectif est de quantifier le compromis entre efficacité de compression et préservation de la qualité de recherche.

**Méthodologie :**
- Compression des images avec les quatre méthodes sélectionnées
- Mesure des métriques de qualité (PSNR, SSIM) et d'efficacité (ratio de compression, temps)
- Évaluation de l'impact sur la précision du moteur de recherche par similarité visuelle
- Analyse comparative des résultats pour identifier les approches optimales selon le contexte d'usage


In [1]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import time
import warnings
warnings.filterwarnings('ignore')

sys.path.append('clean_version')
from compression_engine import CompressionEvaluator
from search_engine import ImageSearchEngine
from analysis_tools import CompressionImpactAnalyzer

print("Analyse de compression d'images pour moteur de recherche")
print("=" * 55)

evaluator = CompressionEvaluator()
classiques = [m for m in evaluator.methods if m.name.startswith(('PNG', 'JPEG'))]
modernes = [m for m in evaluator.methods if m.name.startswith(('Haar', 'DCT'))]

print(f"Méthodes configurées: {[m.name for m in evaluator.methods]}")
print(f"Configuration: {len(classiques)} méthodes classiques, {len(modernes)} méthodes modernes")


Analyse de compression d'images pour moteur de recherche
Méthodes configurées: ['PNG_Lossless', 'JPEG_Q85', 'Haar_T5.0', 'DCT_Q50']
Configuration: 2 méthodes classiques, 2 méthodes modernes


In [2]:
analyzer = CompressionImpactAnalyzer(data_path="data")

print("Début de l'analyse de compression")
print("Durée estimée : 7 minutes")
print()

start_time = time.time()

results = analyzer.run_complete_analysis(
    max_images=15,
    num_queries=10
)

total_time = time.time() - start_time

print(f"\nAnalyse terminée en {total_time:.1f}s")
print(f"Résultats générés: {len(results)} datasets")


Début de l'analyse de compression
Durée estimée : 2-3 minutes

ANALYSE COMPLETE - COMPRESSION POUR RECHERCHE D'IMAGES
Configuration du moteur de recherche baseline...
Dataset train: 2158 images, 11520 features
Index construit: 2158 images
Catégories: ['mountain-peaks' 'mountains' 'snow']
Features: 11520 dimensions
Baseline prêt: 2158 images

Sélection échantillon de 15 images...
Échantillon: 15 images
Évaluation des méthodes de compression...
   Progress: 1/15
   Progress: 6/15
   Progress: 11/15
60 mesures de compression collectées

Résumé compression par méthode:
              compression_ratio    psnr   ssim  compression_time  \
method                                                             
DCT_Q50                   0.046  31.571  0.989             4.915   
Haar_T5.0                 0.033  11.166  0.297             0.034   
JPEG_Q85                  1.452  39.763  0.998             0.001   
PNG_Lossless              0.268     inf  1.000             0.076   

              decom

In [ ]:
report = analyzer.generate_summary_report(results)
print(report)

timestamp = time.strftime('%Y%m%d_%H%M%S')

with open(f'results/rapport_final_compression_{timestamp}.txt', 'w', encoding='utf-8') as f:
    f.write(report)

if 'compression_evaluation' in results:
    results['compression_evaluation'].to_csv(f'results/compression_results_{timestamp}.csv', index=False)
    print(f"Sauvegarde compression: compression_results_{timestamp}.csv")

if 'search_impact' in results:
    results['search_impact'].to_csv(f'results/search_impact_{timestamp}.csv', index=False)  
    print(f"Sauvegarde impact recherche: search_impact_{timestamp}.csv")

print(f"Rapport final: rapport_final_compression_{timestamp}.txt")
print("\nAnalyse terminée - Fichiers de résultats générés")


RAPPORT D'ANALYSE - COMPRESSION POUR RECHERCHE D'IMAGES
Généré: 2025-07-07 12:54:12

CRITÈRES ÉVALUÉS:
- Cohérence choix approches (2 classiques + 2 modernes)
- Qualité compression (PSNR, SSIM, ratio)
- Performance compression (temps)
- Impact sur recherche d'images
- Comparaison objective des approches

RÉSULTATS COMPRESSION
------------------------------

DCT_Q50:
  • Ratio: 0.05x
  • PSNR: 31.6 dB
  • SSIM: 0.989
  • Temps: 4.915s

Haar_T5.0:
  • Ratio: 0.03x
  • PSNR: 11.2 dB
  • SSIM: 0.297
  • Temps: 0.034s

JPEG_Q85:
  • Ratio: 1.45x
  • PSNR: 39.8 dB
  • SSIM: 0.998
  • Temps: 0.001s

PNG_Lossless:
  • Ratio: 0.27x
  • PSNR: inf dB
  • SSIM: 1.000
  • Temps: 0.076s

IMPACT SUR RECHERCHE D'IMAGES
-----------------------------------

PNG_Lossless:
  • Précision baseline: 0.700
  • Précision compressée: 0.500
  • Perte précision: 0.200

JPEG_Q85:
  • Précision baseline: 0.700
  • Précision compressée: 0.400
  • Perte précision: 0.300

Haar_T5.0:
  • Précision baseline: 0.700
  • P